# Required imports

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

In [2]:
def load_dataset_json(path):
    df = pd.read_json(path)
    print(df.info())
    return df

In [3]:
%run kg-extraction.py

/home/manish/miniconda3/envs/askBert/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/manish/miniconda3/envs/askBert/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/manish/miniconda3/envs/askBert/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/manish/miniconda3/envs/askBert/lib

In [4]:
class Args:
	def __init__(self, args):
		self.length = args['length']
		self.batch_size = args['batch_size']
		self.temperature = args['temperature']
		self.model_name = args['model_name']
		self.seed = args['seed']
		self.nsamples = args['nsamples']
		self.cutoffs = args['cutoffs']
		self.write_sfdp = args['write_sfdp']
		self.random = args['random']
                
args = Args({
	'length': 10,
	'batch_size': 1,
	'temperature': 0.5,
	'model_name' : '117M',
	'seed' : 0,
	'nsamples' : 50,
	'cutoffs' : '11.5 15 12',
	'write_sfdp': False,
	'random': False
})

In [5]:
world = World(args)

In [6]:
def get_node_id(graph, node_name, game_id, counter_id):
	if graph.has_node(node_name):
		return graph.nodes[node_name]['id'], counter_id
	else:
		return f'{game_id}_{counter_id}', counter_id + 1

In [7]:
game_id_mappings = {
	'TESO': "The Elder Scrolls: Oblivion",
	'TESS': "The Elder Scrolls: Skyrim",
	'BG1': "Baldur's Gate 1",
	'BG2': "Baldur's Gate 2",
	'MC': "Minecraft",
	'TL2': "Torchlight II"
}

In [8]:
all_quests = []
id_start = 1
edge_id_start = 1
extended_desc = []
skipped_quests = {}

In [9]:
go_to_alternatives = ['Go to', 'Reach', 'Travel to', 'Arrive at', 'Find your way to', 'Head to', 'Make your way to', 'Proceed to']

def get_full_kg(df, id_start, all_quests, extended_desc):
    skipped = [] # to store the ids of the quests that were skipped

    full_game_graph = nx.Graph() # main graph to store complete KG of the game
    total = df.shape[0]

    node_counter = 1
    for item in df.iterrows():
        print(f'\rProcessing {item[0]+1}/{total}', end='')

        item_data = item[1]

        all_characters = []
        all_locations = []
        all_objects = []

        tasks = []
        
        game_id = item_data['game']
        description = ''

        # clear the current context graph
        current_quest_graph = nx.Graph() # graph to store the current quest KG
        
        # add the quest giver as a character
        quest_giver_name = ''
        quest_giver_loc = ''
        if item_data['quest_giver'] != None:
            char = item_data['quest_giver']
            if char['name'] != 'NONE' and char['name'] != '':
                quest_giver_name = char['name']
                node_id, node_counter = get_node_id(full_game_graph, quest_giver_name, game_id, node_counter)                
                current_quest_graph.add_node(quest_giver_name, type='character', description=char['description'], id=node_id)
                all_characters.append(quest_giver_name)
                
                description += f"I am {quest_giver_name}. I am {char['description']}. "
                if char['location'] != '':
                    quest_giver_loc = char['location']
                    if not current_quest_graph.has_node(quest_giver_loc):
                        node_id, node_counter = get_node_id(full_game_graph, quest_giver_loc, game_id, node_counter) 
                        
                        current_quest_graph.add_node(quest_giver_loc, type='location', description=quest_giver_loc, id=node_id)
                        all_locations.append(quest_giver_loc)
                        description += f"{quest_giver_loc} is a location. "
                    current_quest_graph.add_edge(quest_giver_name, quest_giver_loc, label='present_in', id=f'{game_id}_E{edge_id_start}')
                    edge_id_start += 1
                    description += f"{quest_giver_name} is present in {quest_giver_loc}. "

        # add locations from related quest locations
        if item_data['locations'] != None:
            for loc in item_data['locations']:
                if loc['name'] != 'NONE' and loc['name'] != '':
                    node_id, node_counter = get_node_id(full_game_graph, loc['name'], game_id, node_counter) 
                    current_quest_graph.add_node(loc['name'], type='location', description=loc['description'], id=node_id)
                    all_locations.append(loc['name'])
                    description += f"{loc['name']} is a location. {loc['name']} is {loc['description']}. "

        # add locations from quest initial locations
        if item_data['first_task_locations'] != None:
            for loc in item_data['first_task_locations']:
                if loc['name'] != 'NONE':
                    first_task_loc_name = loc['name']
                    if first_task_loc_name == '':
                        first_task_loc_name = loc['description']
                        description += f"{loc['name']} is a location. "
                    else:
                        description += f"{loc['name']} is a location. {loc['name']} is {loc['description']}. "

                    node_id, node_counter = get_node_id(full_game_graph, first_task_loc_name, game_id, node_counter)
                    current_quest_graph.add_node(first_task_loc_name, type='location', description=loc['description'], id=node_id)
                    all_locations.append(loc['name'])

                    if quest_giver_loc != first_task_loc_name:
                        tasks.append(f"{random.choice(go_to_alternatives)} {first_task_loc_name}.")
        
        # add characters from related quest characters
        if item_data['characters'] != None:
            for char in item_data['characters']:
                if char['name'] != 'NONE' and char['name'] != '':
                    node_id, node_counter = get_node_id(full_game_graph, char['name'], game_id, node_counter)
                    current_quest_graph.add_node(char['name'], type='character', description=char['description'], id=node_id)
                    all_characters.append(char['name'])
                    if char['location'] != '':
                        if not current_quest_graph.has_node(char['location']):
                            node_id, node_counter = get_node_id(full_game_graph, char['location'], game_id, node_counter)
                            current_quest_graph.add_node(char['location'], type='location', description=char['location'], id=node_id)
                            all_locations.append(char['location'])
                            description += f"{char['location']} is a location. "
                        current_quest_graph.add_edge(char['name'], char['location'], label='present_in', id=f'{game_id}_E{edge_id_start}')
                        edge_id_start += 1
                        description += f"{char['name']} is present in {char['location']}. {char['name']} is {char['description']}. "
        
        # add characters from quest enemies
        if item_data['enemies'] != None:
            for char in item_data['enemies']:
                if char['name'] != 'NONE':
                    enemy_name = char['name']
                    if enemy_name == '':
                        enemy_name = char['description']
                        description += f"{enemy_name} are enemies. "
                    else:
                        description += f"{enemy_name} are {char['description']}. {enemy_name} are enemies. "
                    node_id, node_counter = get_node_id(full_game_graph, enemy_name, game_id, node_counter)
                    current_quest_graph.add_node(enemy_name, type='character', description=f"[Enemy] {char['description']}", id=node_id)
                    all_characters.append(enemy_name)
        
        # add objects from quest items
        if item_data['items'] != None:
            for obj in item_data['items']:
                if obj['name'] != 'NONE':
                    if obj['name'] != '':
                        obj_name = obj['name']
                        description += f"{obj_name} is an object. {obj['name']} is {obj['description']}. "
                    else:
                        obj_name = obj['description']
                        description += f"{obj_name} is an object. "
                    node_id, node_counter = get_node_id(full_game_graph, obj_name, game_id, node_counter)
                    current_quest_graph.add_node(obj_name, type='object', description=obj['description'], id=node_id)
                    all_objects.append(obj_name)
                    if quest_giver_name != '':
                        current_quest_graph.add_edge(quest_giver_name, obj_name, label='held_by', id=f'{game_id}_E{edge_id_start}')
                        edge_id_start += 1

        quest_description = item_data['description']
        description += '\n' + quest_description
        world.input_text = description
        extended_desc.append(f'This is a quest from {game_id_mappings[game_id]}\n {description}')

        
        # generated_entites = world.generate()
        # for entity in generated_entites:
        #     obj_name, obj_type = entity
        #     node_id, node_counter = get_node_id(full_game_graph, obj_name, game_id, node_counter)
        #     current_quest_graph.add_node(obj_name, type=obj_type, description='', id=node_id)

        # skip the quest if no nodes were added
        if len(current_quest_graph.nodes()) == 0:
            skipped.append((item[0], item_data['name']))
            continue
        
        world.load_from_graph(current_quest_graph)
        edge_id_start = world.autocomplete(f'{game_id}_E', edge_id_start)

        current_quest_graph = world.graph
        full_game_graph = nx.compose(full_game_graph, current_quest_graph)
        
        plots = []
        if item_data.get('motivation') != None:
            plots.append(f"{quest_giver_name} wants {item_data['motivation']}")
        plots.extend(item_data['tools'])
        if 'NONE' in plots:
            plots.remove('NONE')
        if '' in plots:
            plots.remove('')

        if plots == []:
            skipped.append((item[0], item_data['name']))
            continue

        quest_kb = {}

        for node in current_quest_graph.nodes():
            node_desc = current_quest_graph.nodes[node]['description']
            node_type = current_quest_graph.nodes[node]['type']
            quest_kb[current_quest_graph.nodes[node]['id']] = [node, node_desc]
            rels_to_add = [['type', node_type]]
            edges = [e for e in current_quest_graph.edges(node)]
            for edge in edges:
                conn_entity = edge[1]
                rel = current_quest_graph[node][conn_entity]['label']
                if rel == "NA":
                    current_quest_graph.remove_edge(node, conn_entity)
                    continue
                conn_entity_type = current_quest_graph.nodes[conn_entity]['type']
                if node_type == 'location' and conn_entity_type in ['character', 'object']:
                    continue
                if node_type == 'character' and conn_entity_type == 'object':
                    continue
                rels_to_add.append([rel, conn_entity])
                current_quest_graph.remove_edge(node, conn_entity)
            quest_kb[current_quest_graph.nodes[node]['id']].append(rels_to_add)
        
        tasks.extend(item_data['first_tasks'])
        tasks.append(item_data['objective'])

        all_quests.append({
            'id': id_start, 
            'game' : item_data['game'], 
            'kbs' : quest_kb, 
            'plots' : plots, 
            'quest' : {
                'title' : item_data['name'], 
                'objective': item_data['objective'],
                'tasks' : tasks, 
                'description' : quest_description
            }
        })
        id_start += 1

    return full_game_graph, skipped

# Data gathered by Vartinen et al.

From `git@github.com:svartinen/gpt2-quest-descriptions.git`

Värtinen, Susanna, et al. "Generating Role-Playing Game Quests With GPT Language Models." IEEE Transactions on Games, 12 Dec. 2022, pp. 1-12, doi:10.1109/TG.2022.3228480.

In [10]:
vartinen_base = '/home/manish/thesis-implementations/data/VartinenQuests/data_sets/'

## Elder Scrolls Oblivion dataset

In [11]:
oblivion_df = load_dataset_json(vartinen_base + 'TES/quests_oblivion.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  215 non-null    object
 1   objective             215 non-null    object
 2   first_tasks           215 non-null    object
 3   first_task_locations  215 non-null    object
 4   quest_giver           215 non-null    object
 5   reward                215 non-null    object
 6   locations             96 non-null     object
 7   characters            155 non-null    object
 8   items                 68 non-null     object
 9   tools                 215 non-null    object
 10  description           215 non-null    object
 11  groups                123 non-null    object
 12  enemies               4 non-null      object
dtypes: object(13)
memory usage: 22.0+ KB
None


In [12]:
oblivion_df = oblivion_df.drop(['groups'], axis=1)
oblivion_df['game'] = 'TESO'
oblivion_df = oblivion_df.astype(object).where(pd.notnull(oblivion_df),None)
oblivion_df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,locations,characters,items,tools,description,enemies,game
0,The Coldest Sleep,receive Silencer's next contract after killing...,[assassinate Havilstein Hoar-Blood],"[{'name': '', 'description': 'a small campsite...","{'name': 'Lucien Lachance', 'description': 'an...","[{'name': '', 'description': 'gold', 'amount':...","[{'name': 'Nornal', 'description': 'an ancient...","[{'name': 'Silencer', 'description': 'an assas...","[{'name': '""Dead Drop Orders #6""', 'descriptio...","[""Dead Drop Orders #6"" is addressed toward Sil...",Dead Drop Orders #6\nby Lucien Lachance\n\nAlv...,None,TESO
1,Buying a House in Skingrad,purchase Rosethorn Hall,"[give 25,000 septims to Shum gro-Yarug]","[{'name': 'Skingrad', 'description': 'a city'}]","{'name': 'Shum gro-Yarug', 'description': 'the...","[{'name': 'Rosethorn Hall', 'description': 'a ...",None,"[{'name': 'Vandorallen Trebatius', 'descriptio...",None,[the player told Shum gro-Yarug that they want...,"I'm Shum gro-Yarug, the Count's butler.\nSo yo...",None,TESO
2,Buying a house in Chorrol,purchase Arborwatch,"[give 20,000 gold to Countess Valga]","[{'name': 'Chorrol', 'description': 'a city'}]","{'name': 'Countess Valga', 'description': 'a f...","[{'name': 'Arborwatch', 'description': 'a gorg...","[{'name': 'Northern Goods and Trade', 'descrip...","[{'name': 'Seed-Neeus', 'description': 'a fema...",None,[the last tenant of Arborwatch has cleaned the...,It is a pleasure to meet you. I am Countess Va...,None,TESO
3,Falcar's Recommendation,retrieve the Ring of Burden for Falcar,[get the key to the locked well from Deetsan],"[{'name': 'NONE', 'description': ''}]","{'name': 'Falcar', 'description': 'a rude mage...","[{'name': '', 'description': 'Falcar's recomme...","[{'name': 'Arcane University', 'description': ...","[{'name': 'Deetsan', 'description': 'a woman',...","[{'name': 'Ring of Burden', 'description': 'a ...",[the player is an Associate of the mages' guil...,"What is it, Associate? Don't tell me you're he...",None,TESO
4,Caught in the Hunt,find Aleron Loche,[speak with Kurdan gro-Dragol],"[{'name': 'Lonely Suitor Lodge', 'description'...","{'name': 'Ursanne', 'description': 'a worried ...","[{'name': '', 'description': 'anything', 'amou...","[{'name': 'Imperial City', 'description': 'a n...","[{'name': 'Aleron Loche', 'description': 'Ursa...","[{'name': '', 'description': 'gold', 'amount':...",[Aleron Loche wasted Ursanne and his hard earn...,I'm Ursanne. I'm sorry to impose upon you like...,None,TESO


In [13]:
tes_obv_kg, tes_obv_skipped = get_full_kg(oblivion_df, id_start, all_quests, extended_desc)
skipped_quests['TESO'] = tes_obv_skipped
nx.write_gml(tes_obv_kg, "TESOblivion_KG.gml", stringizer=None)

Processing 215/215

## Elder Scrolls Skyrim dataset

In [14]:
skyrim_df = load_dataset_json(vartinen_base + 'TES/quests_skyrim.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  389 non-null    object
 1   objective             389 non-null    object
 2   first_tasks           389 non-null    object
 3   first_task_locations  389 non-null    object
 4   quest_giver           389 non-null    object
 5   reward                389 non-null    object
 6   items                 104 non-null    object
 7   tools                 389 non-null    object
 8   description           389 non-null    object
 9   locations             177 non-null    object
 10  groups                162 non-null    object
 11  characters            248 non-null    object
 12  enemies               4 non-null      object
dtypes: object(13)
memory usage: 39.6+ KB
None


In [15]:
skyrim_df = skyrim_df.drop(['groups'], axis=1)
skyrim_df['game'] = 'TESS'
skyrim_df = skyrim_df.astype(object).where(pd.notnull(skyrim_df),None)
skyrim_df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,items,tools,description,locations,characters,enemies,game
0,Surgery,change your appearance,"[give 1,000 gold to the flesh sculptor for her...","[{'name': 'NONE', 'description': ''}]","{'name': '', 'description': 'the flesh sculpto...","[{'name': '', 'description': 'a new appearance...","[{'name': '', 'description': 'gold', 'amount':...",[NONE],"Are you interested in my services? I warn you,...",None,None,None,TESS
1,Protecting the Bloodline,destroy a vampire before it turns its bandit a...,[kill the vampire bandit],"[{'name': 'NONE', 'description': ''}]","{'name': 'Fura Bloodmouth', 'description': 'a ...","[{'name': 'NONE', 'description': '', 'amount':...",None,[NONE],A feral-blooded has holed up with a group of b...,None,None,None,TESS
2,Unseen Visions,"read ""Blood""",[perform the ritual of the Ancestor Moth to ga...,"[{'name': '', 'description': 'an Ancestor Glad...","{'name': 'Dexion Evicus', 'description': 'a mo...","[{'name': '', 'description': 'the answers the ...","[{'name': '""Blood""', 'description': 'an Elder ...",[the ritual of the Ancestor Moth: the player m...,Scattered across Tamriel are secluded location...,"[{'name': 'Ancestor Glades', 'description': 's...",None,None,TESS
3,Prophet,find a way to read Serana's Elder Scroll,[ask the wizards about reading Elder Scrolls],"[{'name': 'College of Winterhold', 'descriptio...","{'name': 'Serana', 'description': 'a woman', '...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Elder Scrolls', 'description': 'mys...",[Serana has a large Elder Scroll on her back: ...,"So, in case you didn't notice the giant thing ...","[{'name': 'Cyrodiil', 'description': 'a provin...",None,None,TESS
4,Rescue,rescue Farkas from a group of necromancers,[kill the vampire leading the group of necroma...,"[{'name': 'Morvunskar', 'description': 'a necr...","{'name': 'Florentius Baenius', 'description': ...","[{'name': '', 'description': 'an enchanted ite...",None,[NONE],"Arkay has some grave news, friend. Your friend...",None,"[{'name': 'Farkas', 'description': 'a man, and...",None,TESS


In [16]:
tes_sky_kg, tes_sky_skipped = get_full_kg(skyrim_df, id_start, all_quests, extended_desc)
skipped_quests['TESS'] = tes_sky_skipped
nx.write_gml(tes_sky_kg, "TESSkyrim_KG.gml", stringizer=None)

Processing 389/389

## Torchlight 2 dataset

In [17]:
tl2df = load_dataset_json(vartinen_base + 'TL2/quests_TL2.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  80 non-null     object
 1   objective             80 non-null     object
 2   first_tasks           80 non-null     object
 3   first_task_locations  80 non-null     object
 4   quest_giver           80 non-null     object
 5   reward                80 non-null     object
 6   characters            39 non-null     object
 7   tools                 80 non-null     object
 8   description           80 non-null     object
 9   locations             31 non-null     object
 10  items                 33 non-null     object
 11  enemies               6 non-null      object
 12  groups                34 non-null     object
dtypes: object(13)
memory usage: 8.2+ KB
None


In [18]:
tl2df = tl2df.drop(['groups'], axis=1)
tl2df['game'] = 'TL2'
tl2df = tl2df.astype(object).where(pd.notnull(tl2df),None)
tl2df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,characters,tools,description,locations,items,enemies,game
0,Hidden Costs,bring enchanting tools to Baldrick the Enchanter,[search Path of the Honored Dead for a Beornen],"[{'name': 'Path of the Honored Dead', 'descrip...","{'name': 'Baldrick the Enchanter', 'descriptio...","[{'name': '', 'description': 'the player's fir...","[{'name': 'Beornen', 'description': 'a male be...",[Baldrick the Enchanter can't enchant anything...,"Welcome! Welcome, my friend. I have numerous e...",None,None,None,TL2
1,Little Lost Ones,help Bellethe find Finnas and their precious a...,[find Finnas],"[{'name': 'Widow's Veil', 'description': 'a ca...","{'name': 'Bellethe', 'description': 'a worried...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Finnas', 'description': 'Bellethe's...",[Bellethe and Finnas are homesteaders with a s...,"Please, help me! Our dear little ones wandered...","[{'name': 'Estherian Enclave', 'description': ...",None,None,TL2
2,The Thing,destroy the Thing before it infects the whole ...,[find the Thing],"[{'name': 'Norsk Leiren', 'description': 'a co...","{'name': 'Blair', 'description': 'a researcher...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Bennings', 'description': 'one of B...","[the Thing got in Bennings, but then Fuchs dis...","""Blair's Notes""\nby Blair\n\nIt was in the ice...",None,"[{'name': '""Blair's Notes""', 'description': 'n...","[{'name': 'Thing', 'description': 'a dangerous...",TL2
3,Bounty: Sturmbeorn!,alleviate the threat of the Sturmbeorn,[kill 10 Sturmbeorn],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': '', 'description': 'bounty', 'amount...",None,[NONE],BOUNTY!\nThe Sturmbeorn raiders are a threat t...,None,"[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Sturmbeorn', 'description': 'savage...",TL2
4,Bounty: Warbeasts!,help colonists by thinning out packs of Warbeasts,[kill 10 Warbeasts],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': 'NONE', 'description': '', 'amount':...",None,[NONE],BOUNTY!\nCommander Vale of the Vanquisher Corp...,"[{'name': 'Estheria', 'description': 'a countr...","[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Warbeasts', 'description': 'dangero...",TL2


In [19]:
tl2_kg, tl2_skipped = get_full_kg(tl2df, id_start, all_quests, extended_desc)
skipped_quests['TL2'] = tl2_skipped
nx.write_gml(tl2_kg, "Torchlight2_KG.gml", stringizer=None)

Processing 80/80

## Baldur's Gate I Dataset

In [20]:
bg1df = load_dataset_json(vartinen_base + 'BG/quests_BG.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  100 non-null    object
 1   objective             100 non-null    object
 2   first_tasks           100 non-null    object
 3   first_task_locations  100 non-null    object
 4   quest_giver           100 non-null    object
 5   reward                100 non-null    object
 6   tools                 100 non-null    object
 7   description           100 non-null    object
 8   items                 25 non-null     object
 9   characters            58 non-null     object
 10  groups                11 non-null     object
 11  locations             31 non-null     object
 12  enemies               4 non-null      object
dtypes: object(13)
memory usage: 10.3+ KB
None


In [21]:
bg1df = bg1df.drop(['groups'], axis=1)
bg1df['game'] = 'BG1'
bg1df = bg1df.astype(object).where(pd.notnull(bg1df),None)
bg1df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,tools,description,items,characters,locations,enemies,game
0,A Bard's Request,bring an authoritative history of the Unicorn ...,[find an authoritative history of the Unicorn ...,"[{'name': 'NONE', 'description': ''}]","{'name': 'Rinnie', 'description': 'a bard and ...","[{'name': 'Scroll of Protection From Poison', ...",[NONE],"""Sweet waters of the forest\nflow through each...",None,None,None,None,BG1
1,A Book for Firebead,"bring ""The History of the Fateful Coin"" to Fir...","[find ""The History of the Fateful Coin""]","[{'name': '', 'description': 'one of the local...","{'name': 'Firebead Elvenhair', 'description': ...","[{'name': '""History of the Dead Three""', 'desc...",[NONE],"Hello, I am Firebead Elvenhair: an elderly mag...","[{'name': '""The History of the Fateful Coin""',...",None,None,None,BG1
2,A Child in the Lighthouse,save Ardrouine's little son from worgs,[go to the abandoned lighthouse],"[{'name': '', 'description': 'abandoned lighth...","{'name': 'Ardrouine', 'description': 'a distre...","[{'name': '', 'description': 'coins', 'amount'...",[NONE],"Please help me, I am just poor Ardrouine!\nI d...",None,None,None,None,BG1
3,A Contract Killing,assassinate Cyrdemac,[find Cyrdemac],"[{'name': 'Elfsong Tavern', 'description': 'a ...","{'name': 'Areana', 'description': 'a mysteriou...","[{'name': '', 'description': 'gold', 'amount':...",[NONE],Perhaps... perhaps you are the type of people ...,None,"[{'name': 'Cyrdemac', 'description': 'a man wh...",None,None,BG1
4,Albert and His Dog,bring Rufie to Albert,[find Rufie],"[{'name': 'NONE', 'description': ''}]","{'name': 'Albert', 'description': 'a young boy...","[{'name': '', 'description': 'Rufie's another ...",[Albert gives the player Rufie's chew toy to d...,Hi! I'm Albert.\n'Scuse me. I ever so sowwy to...,None,"[{'name': 'Rufie', 'description': 'Albert's pe...",None,None,BG1


In [22]:
bg1_kg, bg1_skipped = get_full_kg(bg1df, id_start, all_quests, extended_desc)
skipped_quests['BG1'] = bg1_skipped
nx.write_gml(bg1_kg, "BaldursGate1_KG.gml", stringizer=None)

Processing 100/100

## Baldur's Gate II Dataset

In [23]:
bg2df = load_dataset_json(vartinen_base + 'BG/quests_BGII.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  94 non-null     object
 1   objective             94 non-null     object
 2   first_tasks           94 non-null     object
 3   first_task_locations  94 non-null     object
 4   quest_giver           94 non-null     object
 5   reward                94 non-null     object
 6   characters            70 non-null     object
 7   tools                 94 non-null     object
 8   description           94 non-null     object
 9   items                 15 non-null     object
 10  groups                35 non-null     object
 11  locations             27 non-null     object
 12  enemies               4 non-null      object
dtypes: object(13)
memory usage: 9.7+ KB
None


In [24]:
bg2df = bg2df.drop(['groups'], axis=1)
bg2df['game'] = 'BG2'
bg2df = bg2df.astype(object).where(pd.notnull(bg2df),None)
bg2df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,characters,tools,description,items,locations,enemies,game
0,A Mage Imprisoned?,free Vithal,[find Vithal],"[{'name': '', 'description': 'in the cavern no...","{'name': 'Uder Mordin', 'description': 'a man ...","[{'name': '', 'description': 'Vithal was tryin...","[{'name': 'Vithal', 'description': 'a powerful...",[NONE],"I would be careful in this place, if I were yo...",None,None,None,BG2
1,A Mage's Proposal,help Vithal breach the walls between dimensions,[recover Vithal's book of rituals],"[{'name': '', 'description': 'a svirfneblin vi...","{'name': 'Vithal', 'description': 'a mage and ...","[{'name': '', 'description': 'items for the pl...",None,"[the player released Vithal from imprisonment,...","Ahh! Thank you for releasing me. I'm Vithal, a...","[{'name': '', 'description': 'Vithal's book of...",None,None,BG2
2,Animal trouble in Trademeet,investigate why forest animals are attacking T...,[go talk with the High Merchant],"[{'name': 'Trademeet', 'description': 'a town'}]","{'name': 'Flydian', 'description': 'a messenge...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'High Merchant', 'description': 'the...","[Flydian was sent by the High Merchant, druids...","I am Flydian, a messenger sent by the High Mer...",None,None,None,BG2
3,Anomen Returns Home After His Sister's Death,investigate the murder of Anomen's sister,[go to Lord Cor's home with Anomen],"[{'name': 'Government District', 'description'...","{'name': 'Anomen', 'description': 'a warrior p...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Lord Cor', 'description': 'a noblem...",[NONE],"I am Anomen, a warrior priest.\nI must discove...",None,"[{'name': 'Athkatla', 'description': 'a city'}]",None,BG2
4,Gain the services of Sir Sarles for the Temple...,convince Sir Sarles to sculpt a statue for the...,[find Sir Sarles],"[{'name': 'Jysstev estate', 'description': 'in...","{'name': 'Dawnbringer Sain', 'description': 'a...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Lathander', 'description': 'the goo...",[Dawnbringer Sain is supposed to acquire art f...,"Greetings, I am known as Dawnbringer Sain, a p...",None,"[{'name': 'Athkatla', 'description': 'a city'}]",None,BG2


In [25]:
bg2_kg, bg2_skipped = get_full_kg(bg2df, id_start, all_quests, extended_desc)
skipped_quests['BG2'] = bg2_skipped
nx.write_gml(bg2_kg, "BaldursGate2_KG.gml", stringizer=None)

Processing 94/94

## Minecraft (Hand-crafted) Dataset

In [26]:
mc_df = load_dataset_json(vartinen_base + 'MC/quests_MC.json')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  100 non-null    object
 1   objective             100 non-null    object
 2   first_tasks           100 non-null    object
 3   first_task_locations  100 non-null    object
 4   quest_giver           100 non-null    object
 5   reward                100 non-null    object
 6   items                 25 non-null     object
 7   motivation            100 non-null    object
 8   tools                 100 non-null    object
 9   description           100 non-null    object
 10  groups                7 non-null      object
 11  characters            13 non-null     object
 12  locations             13 non-null     object
 13  enemies               3 non-null      object
dtypes: object(14)
memory usage: 11.1+ KB
None


In [27]:
mc_df = mc_df.drop(['groups'], axis=1)
mc_df['game'] = 'MC'
mc_df = mc_df.astype(object).where(pd.notnull(mc_df),None)
mc_df.head()

,name,objective,first_tasks,first_task_locations,quest_giver,reward,items,motivation,tools,description,characters,locations,enemies,game
0,Catch of the Day,help Thomas gather fish for his village,[catch 16 fish],"[{'name': 'Salmon Steps', 'description': 'a ne...","{'name': 'Thomas', 'description': 'a concerned...","[{'name': '', 'description': 'emeralds', 'amou...","[{'name': '', 'description': 'fish', 'amount':...",comfort,[NONE],"Traveler, I'm worried about my fellow villager...",None,None,None,MC
1,Pests Be Come,take care of the Endermites that have been bot...,[kill Endermites],"[{'name': '', 'description': 'a village'}]","{'name': 'Matt', 'description': 'an annoyed vi...","[{'name': '', 'description': 'golden carrots',...","[{'name': 'Ender Pearls', 'description': 'spaw...",comfort,[NONE],Some incredibly foolish individual's been thro...,None,None,None,MC
2,Waters of Life,help the villagers of Desert Refuge obtain a p...,[build a well in Desert Refuge],"[{'name': 'Desert Refuge', 'description': 'a v...","{'name': 'Jesse', 'description': 'a thirsty vi...","[{'name': '', 'description': 'a bucket of lava...",None,comfort,[the player is a friend of Jesse],Hail friend! It's good to see you. I'd offer y...,None,None,None,MC
3,Gold. GOLD. GOLD!,bring gold ore to Nila,[find some gold ore],"[{'name': '', 'description': 'ore veins deep i...","{'name': 'Nila', 'description': 'a greedy vill...","[{'name': '', 'description': 'melon seeds', 'a...",None,comfort,[the player is an enemy of Nila],"What are YOU doing here! If it was up to me, I...",None,None,None,MC
4,Our Father in the Sky Like the Diamonds,bring 9 diamonds to Paul,[find 9 diamonds],"[{'name': '', 'description': 'ancient ruins, a...","{'name': 'Paul', 'description': 'a cleric', 'l...","[{'name': 'Glowstone Dust', 'description': 'a ...","[{'name': '', 'description': 'diamonds', 'amou...",comfort,[NONE],"My child, my temple could use a donation. I ha...",None,None,None,MC


In [28]:
mc_kg, mc_skipped = get_full_kg(mc_df, id_start, all_quests, extended_desc)
skipped_quests['MC'] = mc_skipped
nx.write_gml(mc_kg, "Minecraft_KG.gml", stringizer=None)

Processing 100/100

## Save dataset

- Randomize all quests
- Split for train, test, val (70:20:10)
- Save as JSONs

In [29]:
import random
import json

In [30]:
def write_to_file(content, filename):
    json_object = json.dumps(content, indent=4)
    with open(filename, "w") as outfile:
        outfile.write(json_object)

In [31]:
# uncomment to get a general json file

for entry in all_quests:
    kbs = []
    for id, v in entry['kbs'].items():
        entity = v[0]
        desc = v[1]
        kb = {'name': entity, 'description': desc, 'id': id, 'relations': []}
        relations = v[2]
        for rel in relations:
            if rel[0] == 'type':
                kb['type'] = rel[1]
            else:
                kb['relations'].append(rel)
        kbs.append(kb)
    entry['kbs'] = kbs

In [32]:
write_to_file(all_quests, 'all_quests.json')

In [33]:
random.shuffle(all_quests)

full_size = len(all_quests)
train_size = int(full_size * 0.8)
test_size = int(full_size * 0.15)

print(f'Total number of quests: {full_size}')
print(f'Train size: {train_size}')
print(f'Test size: {test_size}')
print(f'Validation size: {full_size - (test_size + train_size)}')

Total number of quests: 769
Train size: 615
Test size: 115
Validation size: 39


In [34]:
train_set = all_quests[:train_size]
test_set = all_quests[train_size:train_size+test_size]
val_set = all_quests[train_size+test_size:]

In [35]:
write_to_file(train_set, 'train.json')
write_to_file(test_set, 'test.json')
write_to_file(val_set, 'val.json')

In [36]:
with open('extended_descriptions.txt', 'w') as f:
    f.write('\n\n'.join(extended_desc))

In [37]:
%rm -rf ../../../../data/VartinenFormatted/KGs/*
%mv *.gml ../../../../data/VartinenFormatted/KGs/

%rm -rf ../../../../data/VartinenFormatted/all_quests.json
%mv all_quests.json ../../../../data/VartinenFormatted/

%rm -rf ../../../../data/VartinenFormatted/extended_descriptions.txt
%mv extended_descriptions.txt ../../../../data/VartinenFormatted/

%rm -rf ../../../../data/VartinenFormatted/KGPT/*
%mv {test,train,val}.json ../../../../data/VartinenFormatted/KGPT/

OMP: Info #250: KMP_AFFINITY: pid 166590 tid 166590 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166591 tid 166591 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166592 tid 166592 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166593 tid 166593 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166594 tid 166594 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166595 tid 166595 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166596 tid 166596 thread 0 bound to OS proc set 0-15
OMP: Info #250: KMP_AFFINITY: pid 166598 tid 166598 thread 0 bound to OS proc set 0-15
